<a href="https://colab.research.google.com/github/GiackAloZ/di-playstore/blob/master/play-store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Progetto di DI - Google Play Store: Category Classification

Questo progetto prende in esame un dataset ottenuto dal Google Play Store, dove sono state collezionate le informazioni riguardanti piu' di 10.000 app.

Lo scopo del progetto e' quello di utilizzare semplici tecniche di NLP per classificare una app nella sua categoria (eg. Game, Social, Art & Design, etc...) a partire dal suo nome.

Ho trovato il dataset su Kaggle ed e' consultabile [qui](https://www.kaggle.com/lava18/google-play-store-apps).



## Descrizione del problema e analisi esplorativa

Si deve realizzare un modello che, dato il nome di una app, la classifichi in base alla sua categoria tra le varie disponibili (eg. Game, Social, Art & Design, etc...)

Per prima cosa, importiamo le librerie che ci serviranno.

In [ ]:
# from google.colab import files
# files.upload()
# ! mkdir ~/.kaggle
# ! mv kaggle.json ~/.kaggle
# ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import os

import numpy as np
import pandas as pd
import sklearn as skl
import tensorflow.keras as ks
import wordcloud
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#import kaggle

from sklearn.feature_extraction.text import CountVectorizer

Scarichiamo il dataset da kaggle (se non e' gia' presente nella repo o in Colab)

In [ ]:
! kaggle datasets download -d lava18/google-play-store-apps
! unzip -n google-play-store-apps.zip

Carichiamo i dati dal csv `googleplaystore.csv` come un dataframe pandas e diamo un'occhiata alla sua shape e alle prime 5 righe.

In [ ]:
apps_data = pd.read_csv("googleplaystore.csv")
print(apps_data.shape)
apps_data.head(5)

Ci sono numerose informazioni in questo dataset, ma le colonne che andremo a considerare sono due:

- `App`, ovvero i nomi delle app
- `Category`, la categoria a cui appartiene ogni app

Andiamo a osservare quali possono essere le categorie a cui un'applicazione puo' appartenere.

In [ ]:
apps_data["Category"].value_counts()

C'e' una categoria che probabilemente e' stata inserita per errore, cioe' la categoria identificata con il nome "1.9", perche' ha una sola occorrenza. Andiamo a vedere di cosa si tratta.

In [ ]:
wrong_data = apps_data[apps_data["Category"] == "1.9"]
wrong_data

Sembrerebbe un dato errato, infatti ha tutte le colonne sbagliate o shiftate. Possiamo rimuoverla dal dataset.

In [ ]:
apps_data = apps_data[~apps_data.index.isin(wrong_data.index)].reset_index()

Andiamo ora a visualizzare le categorie in un grafico a barre

In [ ]:
apps_data["Category"].value_counts().plot.bar()

Notiamo che le categorie spaziano da molto popolate (eg. FAMILY o GAME) a scarsamente popolate (eg. COMICS o BEAUTY).
Questo potrebbe causare problemi durante la fase di apprendimento, ma per adesso ci puo' andare bene.

Vogliamo ora farci un'idea di come sono fatti i nomi delle app. Per fare cio', andiamo a concatenare tutti i nomi in una string `text` per poi creare una `cloudword` di tutti i nomi delle app in modo da osservare a colpo d'occhio quali sono le parole che compaiono piu' spesso.

In [ ]:
text = " ".join(apps_data["App"])

In [ ]:
wc = wordcloud.WordCloud(
    width=800, height=800,
    background_color="white", max_words=200
).generate(text)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(wc)

Notiamo subito che le parole piu' utilizzate sono quelle che ci potevamo aspettare, come "App" o "Mobile" oppure anche "Free".